In [1]:
# Import dependencies
import pandas as pd
import os
from sqlalchemy import create_engine
import Resources.config as c
import warnings
warnings.filterwarnings('ignore')
import Resources.config as c

In [2]:
# Import movies, series, and IMDb CSV and create dataframes
movies_csv = os.path.join("Resources", "streaming_movies.csv")
movies_df = pd.read_csv(movies_csv)

series_csv = os.path.join("Resources", "streaming_series.csv")
series_df = pd.read_csv(series_csv)

IMDB_movies = os.path.join("Resources/IMDb_movies.csv")
IMDB_movies_df = pd.read_csv(IMDB_movies, low_memory=False)

In [3]:
# View movies dataframe
movies_df.head()

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0


In [4]:
# View series dataframe
series_df.head()

,Unnamed: 0,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,type
0,0,Breaking Bad,2008,18+,9.5,96%,1,0,0,0,1
1,1,Stranger Things,2016,16+,8.8,93%,1,0,0,0,1
2,2,Money Heist,2017,18+,8.4,91%,1,0,0,0,1
3,3,Sherlock,2010,16+,9.1,78%,1,0,0,0,1
4,4,Better Call Saul,2015,18+,8.7,97%,1,0,0,0,1


In [5]:
# Drop unnamed columns from the movies and series CSVs 
# (used Stack Overflow for this: https://stackoverflow.com/questions/43983622/remove-unnamed-columns-in-pandas-dataframe/43983654)
movies_df.drop(movies_df.columns[movies_df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
series_df.drop(series_df.columns[series_df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [6]:
# Find missing data in the movies and series dataframe
print(movies_df.count())
print("-----------------") # Makes it easier to read
print(series_df.count())

ID                 16744
Title              16744
Year               16744
Age                 7354
IMDb               16173
Rotten Tomatoes     5158
Netflix            16744
Hulu               16744
Prime Video        16744
Disney+            16744
Type               16744
Directors          16018
Genres             16469
Country            16309
Language           16145
Runtime            16152
dtype: int64
-----------------
Title              5611
Year               5611
Age                3165
IMDb               4450
Rotten Tomatoes    1011
Netflix            5611
Hulu               5611
Prime Video        5611
Disney+            5611
type               5611
dtype: int64


In [7]:
# Drop any rows missing data
movies_df.dropna(axis=0, how="any", inplace=True)
series_df.dropna(axis=0, how="any", inplace=True)

print(movies_df.count())
print("---------------")
print(series_df.count())

ID                 3301
Title              3301
Year               3301
Age                3301
IMDb               3301
Rotten Tomatoes    3301
Netflix            3301
Hulu               3301
Prime Video        3301
Disney+            3301
Type               3301
Directors          3301
Genres             3301
Country            3301
Language           3301
Runtime            3301
dtype: int64
---------------
Title              931
Year               931
Age                931
IMDb               931
Rotten Tomatoes    931
Netflix            931
Hulu               931
Prime Video        931
Disney+            931
type               931
dtype: int64


In [8]:
# Find column types in case we need to convert to other type
print(movies_df.dtypes)
print("--------------")
print(series_df.dtypes)

ID                   int64
Title               object
Year                 int64
Age                 object
IMDb               float64
Rotten Tomatoes     object
Netflix              int64
Hulu                 int64
Prime Video          int64
Disney+              int64
Type                 int64
Directors           object
Genres              object
Country             object
Language            object
Runtime            float64
dtype: object
--------------
Title               object
Year                 int64
Age                 object
IMDb               float64
Rotten Tomatoes     object
Netflix              int64
Hulu                 int64
Prime Video          int64
Disney+              int64
type                 int64
dtype: object


In [9]:
# Combine title and year for both movies and series which we will then use to create a unique ID for each film 
# Used code from "https://cmdlinetips.com/2018/11/how-to-join-two-text-columns-into-a-single-column-in-pandas/" to help with this
movies_df["Movie_Title"] = movies_df["Year"].astype(str).str.cat(movies_df["Title"],sep="-")
series_df["Movie_Title"] = series_df["Year"].astype(str).str.cat(series_df["Title"],sep="-")

In [10]:
# Use hash func to create unique ID for movies, then add to dataframe
unique_movie_id = (movies_df['Movie_Title'].apply(hash))
movies_df.insert(0,"Unique_ID", unique_movie_id)
del movies_df["ID"]
del movies_df["Type"]
movies_df.head()

,Unique_ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Directors,Genres,Country,Language,Runtime,Movie_Title
0,3353255878963028353,Inception,2010,13+,8.8,87%,1,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0,2010-Inception
1,-5184210925941368056,The Matrix,1999,18+,8.7,87%,1,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0,1999-The Matrix
2,2721945714620145150,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0,2018-Avengers: Infinity War
3,-6037567714654325504,Back to the Future,1985,7+,8.5,96%,1,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0,1985-Back to the Future
4,1540339058890638920,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0,"1966-The Good, the Bad and the Ugly"


In [11]:
# Use hash func to create unique ID for series, then add to dataframe
unique_id = (series_df['Movie_Title'].apply(hash))
series_df.insert(0,"Unique_ID", unique_id)
del series_df["type"]
series_df.head()

,Unique_ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Movie_Title
0,-4172620574754119178,Breaking Bad,2008,18+,9.5,96%,1,0,0,0,2008-Breaking Bad
1,8846535452306050157,Stranger Things,2016,16+,8.8,93%,1,0,0,0,2016-Stranger Things
2,245040933651337214,Money Heist,2017,18+,8.4,91%,1,0,0,0,2017-Money Heist
3,886905699165934950,Sherlock,2010,16+,9.1,78%,1,0,0,0,2010-Sherlock
4,-1746450592548723018,Better Call Saul,2015,18+,8.7,97%,1,0,0,0,2015-Better Call Saul


In [12]:
# Rename columns so they be can be read in postgres
movies_df = movies_df.rename(columns={"Unique_ID":"unique_id", "Title":"title", "Year":"year", "Age":"age", "IMDb":"imdb", "Rotten Tomatoes":"rotten_tomatoes", 
                                      "Netflix":"netflix", "Hulu":"hulu", "Prime Video":"prime_video", "Disney+":"disney_plus", "Directors":"directors", 
                                      "Genres":"genres", "Country":"country", "Language":"language", "Runtime":"runtime", "Movie_Title":"movie_title"})
series_df = series_df.rename(columns={"Unique_ID":"unique_id", "Title":"title", "Year":"year", "Age":"age", "IMDb":"imdb", "Rotten Tomatoes":"rotten_tomatoes", 
                                      "Netflix":"netflix", "Hulu":"hulu", "Prime Video":"prime_video", "Disney+":"disney_plus", "Movie_Title":"movie_title"})

In [13]:
# Set index to unique_id so this can be our primary key in postgres
movies_df.set_index("unique_id", inplace=True)
series_df.set_index("unique_id", inplace=True)

In [14]:
# View cleaned movies dataframe
movies_df.head()

,title,year,age,imdb,rotten_tomatoes,netflix,hulu,prime_video,disney_plus,directors,genres,country,language,runtime,movie_title
unique_id,,,,,,,,,,,,,,,
3353255878963028353,Inception,2010,13+,8.8,87%,1,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0,2010-Inception
-5184210925941368056,The Matrix,1999,18+,8.7,87%,1,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0,1999-The Matrix
2721945714620145150,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0,2018-Avengers: Infinity War
-6037567714654325504,Back to the Future,1985,7+,8.5,96%,1,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0,1985-Back to the Future
1540339058890638920,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0,"1966-The Good, the Bad and the Ugly"


In [15]:
# View cleaned series dataframe
series_df.head()

,title,year,age,imdb,rotten_tomatoes,netflix,hulu,prime_video,disney_plus,movie_title
unique_id,,,,,,,,,,
-4172620574754119178,Breaking Bad,2008,18+,9.5,96%,1,0,0,0,2008-Breaking Bad
8846535452306050157,Stranger Things,2016,16+,8.8,93%,1,0,0,0,2016-Stranger Things
245040933651337214,Money Heist,2017,18+,8.4,91%,1,0,0,0,2017-Money Heist
886905699165934950,Sherlock,2010,16+,9.1,78%,1,0,0,0,2010-Sherlock
-1746450592548723018,Better Call Saul,2015,18+,8.7,97%,1,0,0,0,2015-Better Call Saul


In [16]:
# View IMDb dataframe
IMDB_movies_df.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,12/26/06,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,"$2,250",NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,8/19/11,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,11/13/12,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,"$45,000",NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,3/6/11,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0


In [17]:
# Select columns we want to keep
imdb_df = IMDB_movies_df[['imdb_title_id','title','original_title','year','genre','duration','country']]
imdb_df.head()

,imdb_title_id,title,original_title,year,genre,duration,country
0,tt0000009,Miss Jerry,Miss Jerry,1894,Romance,45,USA
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,"Biography, Crime, Drama",70,Australia
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,Drama,53,"Germany, Denmark"
3,tt0002101,Cleopatra,Cleopatra,1912,"Drama, History",100,USA
4,tt0002130,L'Inferno,L'Inferno,1911,"Adventure, Drama, Fantasy",68,Italy


In [18]:
# Count rows/find missing data
imdb_df.count()

imdb_title_id     85849
title             85849
original_title    85849
year              85849
genre             85849
duration          85849
country           85785
dtype: int64

In [19]:
# Remove any null values
imdb_df2=imdb_df.dropna()
imdb_df2.count()

imdb_title_id     85785
title             85785
original_title    85785
year              85785
genre             85785
duration          85785
country           85785
dtype: int64

In [20]:
# Add new columns to IMDb dataframe that combines the year and movie title, and for new movie_id
imdb_df2.insert(5,'unique_id','')
imdb_df2.insert(6, 'movie_title','')
imdb_df2.head()

,imdb_title_id,title,original_title,year,genre,unique_id,movie_title,duration,country
0,tt0000009,Miss Jerry,Miss Jerry,1894,Romance,,,45,USA
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,"Biography, Crime, Drama",,,70,Australia
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,Drama,,,53,"Germany, Denmark"
3,tt0002101,Cleopatra,Cleopatra,1912,"Drama, History",,,100,USA
4,tt0002130,L'Inferno,L'Inferno,1911,"Adventure, Drama, Fantasy",,,68,Italy


In [21]:
# Find column types in case we need to convert to other type
imdb_df2.dtypes

imdb_title_id     object
title             object
original_title    object
year               int64
genre             object
unique_id         object
movie_title       object
duration           int64
country           object
dtype: object

In [22]:
# Combine title and year which we will then use to create a unique ID for each film 
# Used code from "https://cmdlinetips.com/2018/11/how-to-join-two-text-columns-into-a-single-column-in-pandas/" to help with this
# Find out what error message means
imdb_df2['movie_title']=imdb_df2['year'].astype(str).str.cat(imdb_df2['title'],sep="-")
imdb_df2.head()

,imdb_title_id,title,original_title,year,genre,unique_id,movie_title,duration,country
0,tt0000009,Miss Jerry,Miss Jerry,1894,Romance,,1894-Miss Jerry,45,USA
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,"Biography, Crime, Drama",,1906-The Story of the Kelly Gang,70,Australia
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,Drama,,1911-Den sorte drøm,53,"Germany, Denmark"
3,tt0002101,Cleopatra,Cleopatra,1912,"Drama, History",,1912-Cleopatra,100,USA
4,tt0002130,L'Inferno,L'Inferno,1911,"Adventure, Drama, Fantasy",,1911-L'Inferno,68,Italy


In [23]:
# Use hash function to generate unique movie id based on movie title
imdb_df2['unique_id']=imdb_df2['imdb_title_id'].apply(hash)
imdb_df2.head()

,imdb_title_id,title,original_title,year,genre,unique_id,movie_title,duration,country
0,tt0000009,Miss Jerry,Miss Jerry,1894,Romance,-409077616621258993,1894-Miss Jerry,45,USA
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,"Biography, Crime, Drama",866313333955130904,1906-The Story of the Kelly Gang,70,Australia
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,Drama,838600562071848784,1911-Den sorte drøm,53,"Germany, Denmark"
3,tt0002101,Cleopatra,Cleopatra,1912,"Drama, History",4145386050906707501,1912-Cleopatra,100,USA
4,tt0002130,L'Inferno,L'Inferno,1911,"Adventure, Drama, Fantasy",-1455654637378694813,1911-L'Inferno,68,Italy


In [24]:
# Rearrange column order so data is properly read into postgres
imdb=imdb_df2.reindex(columns=['unique_id','movie_title','imdb_title_id',
                                  'title','original_title','year',
                                  'genre','duration','country'])
imdb.head()

,unique_id,movie_title,imdb_title_id,title,original_title,year,genre,duration,country
0,-409077616621258993,1894-Miss Jerry,tt0000009,Miss Jerry,Miss Jerry,1894,Romance,45,USA
1,866313333955130904,1906-The Story of the Kelly Gang,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,"Biography, Crime, Drama",70,Australia
2,838600562071848784,1911-Den sorte drøm,tt0001892,Den sorte drøm,Den sorte drøm,1911,Drama,53,"Germany, Denmark"
3,4145386050906707501,1912-Cleopatra,tt0002101,Cleopatra,Cleopatra,1912,"Drama, History",100,USA
4,-1455654637378694813,1911-L'Inferno,tt0002130,L'Inferno,L'Inferno,1911,"Adventure, Drama, Fantasy",68,Italy


In [25]:
# Set index to unique_id so this can be our primary key in postgres
imdb.set_index("unique_id", inplace=True)
imdb

,movie_title,imdb_title_id,title,original_title,year,genre,duration,country
unique_id,,,,,,,,
-409077616621258993,1894-Miss Jerry,tt0000009,Miss Jerry,Miss Jerry,1894,Romance,45,USA
866313333955130904,1906-The Story of the Kelly Gang,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,"Biography, Crime, Drama",70,Australia
838600562071848784,1911-Den sorte drøm,tt0001892,Den sorte drøm,Den sorte drøm,1911,Drama,53,"Germany, Denmark"
4145386050906707501,1912-Cleopatra,tt0002101,Cleopatra,Cleopatra,1912,"Drama, History",100,USA
-1455654637378694813,1911-L'Inferno,tt0002130,L'Inferno,L'Inferno,1911,"Adventure, Drama, Fantasy",68,Italy
...,...,...,...,...,...,...,...,...
-3483546144103097732,2020-Le lion,tt9908390,Le lion,Le lion,2020,Comedy,95,"France, Belgium"
8965474185003017959,2020-De Beentjes van Sint-Hildegard,tt9911196,De Beentjes van Sint-Hildegard,De Beentjes van Sint-Hildegard,2020,"Comedy, Drama",103,Netherlands
-6083446290931633814,2019-Padmavyuhathile Abhimanyu,tt9911774,Padmavyuhathile Abhimanyu,Padmavyuhathile Abhimanyu,2019,Drama,130,India


In [26]:
# Create database connection
imdb_connection = f"postgresql://{c.username}:{c.password}@{c.db_connect_string}"
engine = create_engine(imdb_connection)

In [27]:
# Confirm tables exist
engine.table_names()

['movies', 'series', 'imdb', 'imdb_movies']

In [28]:
# Load dataframes into database
movies_df.to_sql(name="movies", con=engine, if_exists="append", index=True)
series_df.to_sql(name="series", con=engine, if_exists="append", index=True)
imdb.to_sql(name="imdb_movies", con=engine, if_exists="append", index=True, method="multi")

In [29]:
series_df.to_html('series_df_tables.html', max_rows=100)